# **Running the BAO fit for a given dataset and settings**

This notebook shows how to run a BAO fit

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. Single BAO fit

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

# 1. Arguments
class Args:
    def __init__(self):
        self.include_wiggles = "y"
        self.dataset = "DESY6"
        # self.dataset = "DESY6_dec<-23.5"
        # self.dataset = "DESY6_dec>-23.5"
        self.weight_type = 1
        self.mock_id = "mean" # it will not be used...
        self.nz_flag = "fid"
        # self.nz_flag = "clusteringz"
        self.cov_type = "cosmolike"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.delta_theta = 0.2
        self.theta_min = 0.5
        self.theta_max = 5
        self.n_broadband = 3
        self.bins_removed = []
        self.diag_only = "n"
        self.remove_crosscov = "n"
        self.alpha_min = 0.8
        self.alpha_max = 1.2
        self.base_path = None
# class Args:
#     def __init__(self):
#         self.include_wiggles = "y"
#         self.dataset = "DESY6_COLA"
#         self.weight_type = 1 # it will not be used...
#         self.mock_id = "mean"
#         self.nz_flag = "mocks"
#         self.cov_type = "cosmolike"
#         self.cosmology_template = "mice"
#         self.cosmology_covariance = "mice"
#         self.delta_theta = 0.2
#         self.theta_min = 0.5
#         self.theta_max = 5
#         self.n_broadband = 3
#         self.bins_removed = []
#         self.diag_only = "n"
#         self.remove_crosscov = "n"
#         self.alpha_min = 0.8
#         self.alpha_max = 1.2
#         self.base_path = None
# class Args:
#     def __init__(self):
#         self.include_wiggles = "y"
#         self.dataset = "DESIY1_LRG_EZ"
#         self.weight_type = 1 # it will not be used...
#         self.mock_id = "mean"
#         self.nz_flag = "mocks"
#         self.cov_type = "mocks"
#         self.cosmology_template = "desifid"
#         self.cosmology_covariance = "desifid"
#         self.delta_theta = 0.2
#         self.theta_min = 0.5
#         self.theta_max = 8
#         self.n_broadband = 3
#         self.bins_removed = []
#         self.diag_only = "n"
#         self.remove_crosscov = "n"
#         self.alpha_min = 0.8
#         self.alpha_max = 1.2
#         self.base_path = None
args = Args()
args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

fit_galaxybias = True

if fit_galaxybias:
    # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id="mean",
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=0.2,
        theta_min=0.5,
        theta_max=2,
        bins_removed=[],
        diag_only="n",
        remove_crosscov="n",
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    nbins = len(wtheta_data)
    wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])

    # 3. Get the model wtheta (for the linear galaxy bias fit)
    wtheta_model_galaxybias = WThetaModelGalaxyBias(
        include_wiggles="",
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        base_path=args.base_path,
    )

    # 5. Fit the linear galaxy bias
    popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
    galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}

    print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")

else:
    galaxy_bias = {bin_z: 2 for bin_z in range(10)}

# 2. Load data w(theta) and covariance
wtheta_data_covariance = WThetaDataCovariance(
    dataset=args.dataset,
    weight_type=args.weight_type,
    mock_id=args.mock_id,
    nz_flag=args.nz_flag, # this is only needed to get nbins...
    cov_type=args.cov_type,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    bins_removed=args.bins_removed,
    diag_only=args.diag_only,
    remove_crosscov=args.remove_crosscov,
)
theta_data, wtheta_data, cov = wtheta_data_covariance.process()

# 3. Get the model wtheta
wtheta_model = WThetaModel(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    nz_flag=args.nz_flag,
    cosmology_template=args.cosmology_template,
    n_broadband=args.n_broadband,
    galaxy_bias=galaxy_bias,
    base_path=args.base_path,
)

# 4. BAO fit initializer. This basically creates the path to save the results
baofit_initializer = BAOFitInitializer(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    weight_type=args.weight_type,
    mock_id=args.mock_id,
    nz_flag=args.nz_flag,
    cov_type=args.cov_type,
    cosmology_template=args.cosmology_template,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    n_broadband=args.n_broadband,
    bins_removed=args.bins_removed,
    alpha_min=args.alpha_min,
    alpha_max=args.alpha_max,
    verbose=True,
    base_path=args.base_path,
)

# 5. Run the BAO fit
baofit = BAOFit(
    baofit_initializer=baofit_initializer,
    wtheta_model=wtheta_model,
    theta_data=theta_data,
    wtheta_data=wtheta_data,
    cov=cov,
    close_fig=False,
    use_multiprocessing=True,
    n_cpu=256
)
alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 2. Multiple BAO fits (varying settings)

In [ ]:
import numpy as np
from itertools import product
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

dataset_list = ["DESY6"]
# dataset_list = ["DESY6_dec<-23.5", "DESY6_dec>-23.5"]
# dataset_list = ["DESY6_DR1tiles_noDESI"]
# dataset_list = ["DESY6_DR1tiles_DESIonly"]
include_wiggles_list = ["y", "n"]
weight_type_list = [1, 0]
bins_removed_list = [[], [0], [1], [2], [3], [4], [5], [1,2,3,4,5], [0,2,3,4,5], 
                     [0,1,3,4,5], [0,1,2,4,5], [0,1,2,3,5], [0,1,2,3,4]]

for dataset, include_wiggles, weight_type, bins_removed in product(dataset_list, include_wiggles_list, weight_type_list, bins_removed_list):
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = include_wiggles
            self.dataset = dataset
            self.weight_type = weight_type
            self.mock_id = "mean" # it will not be used...
            self.nz_flag = "fid"
            self.cov_type = "cosmolike"
            self.cosmology_template = "planck"
            self.cosmology_covariance = "planck"
            self.delta_theta = 0.2
            self.theta_min = 0.5
            self.theta_max = 5
            self.n_broadband = 3
            self.bins_removed = bins_removed
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    fit_galaxybias = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min=0.5,
            theta_max=2,
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 5. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    else:
        galaxy_bias = {bin_z: 2 for bin_z in range(10)}
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=True,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()
    

## 3. Multiple BAO fits (all mocks)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

mock_id_list = range(1000)

for mock_id in mock_id_list:
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = "DESIY1_LRG_EZ"
            self.weight_type = 1 # it will not be used...
            self.mock_id = mock_id
            self.nz_flag = "mocks"
            self.cov_type = "mocks"
            self.cosmology_template = "desifid"
            self.cosmology_covariance = "desifid"
            self.delta_theta = 0.2
            self.theta_min = 0.5
            self.theta_max = 8
            self.n_broadband = 3
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

    if mock_id == 0:
        fit_galaxybias = True
    else:
        fit_galaxybias = False
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min=0.5,
            theta_max=2,
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 5. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=True,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()
